## 1. 사용하는 도구들 나열해보기

In [ ]:
import pandas as pd # 모욕어 데이터 프레임 가져오기
from transformers import BertTokenizer # 버트로 한국어 텍스트를 토큰화
from konlpy.tag import Okt # 한국어 말머리
from nltk.corpus import stopwords # 필요없는 불용어 제거 시 사용
import nltk # stopwords 다운로드에 사용
from tqdm import tqdm # 작업시간을 확인하는 용도
from sentence_transformers import util, SentenceTransformer # 한국어 텍스트를 임베딩 처리
from sklearn.model_selection import train_test_split # 학습과 테스트
from imblearn.over_sampling import SMOTE # 오버 샘플링 도구
from imblearn.under_sampling import RandomUnderSampler # 언더 샘플링 도구
from sklearn.linear_model import LogisticRegression  # 로지스틱 회귀 모델 
from sklearn.metrics import accuracy_score # 정확도 평가
import joblib # 피클 파일 저장

nltk.download('stopwords') # 한국어 불용어 다운로드

## 2. 데이터 프레임 가져온 후 토큰화 전 처리과정

In [ ]:
df = pd.read_csv('data_split_3.tsv', delimiter='\t') # 98만개 모욕어 데이터 가져오기
df = df.set_index('index')
df = df[~df.duplicated(subset='text')] # 중복 값 제거
df = df.reset_index(drop=True) 
df['text'] = df['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True) # 한국어 내용이 아닌 텍스트 제거
df = df.reset_index(drop=True)
df.info()

In [ ]:
df = df.dropna(subset=['label']) # 라벨 컬럼의 결측치 제거
df['label'] = df['label'].astype(int) # 실수형을 정수형으로 바꾸기

In [ ]:
df = df.dropna() # 다시 결측치 제거
df = df.reset_index(drop=True)
df.info()

## 3. 토큰화 후 임베딩 처리

In [ ]:
tqdm.pandas()

stop_words = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', '되다', '이다', '있다', '없다', '가다', '오다', '보다', '그', '저']
okt = Okt()

def preprocess_text(review, stop_words):
    tokens = okt.morphs(review)  # 형태소 분석으로 토큰화
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)  # 공백으로 연결하여 반환

# 전처리 적용
df['processed_text'] = df['text'].progress_apply(lambda x: preprocess_text(x, stop_words))

In [ ]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2') # 버트 모델 가져오기
df['embedding'] = df['processed_text'].progress_apply(lambda x: model.encode(x).tolist()) # 토큰화 후 가져온 버트 모델 적용

## 4. 샘플링 후 로지스틱 회귀 학습하기

#### 4-1. stratify 매개변수는 데이터셋의 클래스 비율을 고려하여 분할하고 그 매개변수 값이 y라면, 이진 분류(0 or 1)에서 각 레이블 값을 나타내게 됩니다.

In [ ]:
# 언더샘플링 및 오버샘플링 적용
X = df['embedding'].tolist()  # 임베딩 벡터들
y = df['label'].values        # 레이블

# 데이터셋 분할 (Train/Test Split)
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# 언더샘플링 (다수 클래스 줄이기)
rus = RandomUnderSampler(random_state=42) # RandomUnderSampler는 언더 샘플링 도구
X_train_under, y_train_under = rus.fit_resample(X_train, y_train) 

# SMOTE 오버샘플링 (소수 클래스 증강)
smote = SMOTE(random_state=42) # SMOTE는 오버샘플링 도구
X_resampled, y_resampled = smote.fit_resample(X_train_under, y_train_under)

In [ ]:
# 모델 학습 (로지스틱 회귀 사용)
clf = LogisticRegression(random_state=42)  # 로지스틱 회귀 모델 생성
clf.fit(X_resampled, y_resampled)

# 모델 평가
y_pred = clf.predict(X_test)
print(f"테스트 데이터 정확도: {accuracy_score(y_test, y_pred)}")

## 5. 로지스틱 회귀 모델과 버트 모델 저장하기

In [ ]:
# 로지스틱 회귀 모델 저장
joblib.dump(clf, 'logistic_regression_model.pkl')

# S-BERT 모델 저장 (SentenceTransformer)
model.save('sentence_transformer_model')

## 6. 적용시킬 챗봇 모델을 불러온 후 로지스틱 모델과 버트 모델 적용

In [ ]:
health_df = pd.read_csv('health_df.csv') # 건강보건 챗봇 데이터
public_df = pd.read_csv('public_result.csv') # 공공민원 챗봇 데이터

# tqdm과 pandas를 통합
tqdm.pandas()

# 로지스틱 회귀 모델 로드
clf = joblib.load('logistic_regression_model.pkl')

# S-BERT 모델 로드
model = SentenceTransformer('sentence_transformer_model')

In [ ]:
# 모델 임베딩 처리후 적용
public_df['embedding'] = public_df['question'].progress_apply(lambda x: model.encode(x))
health_df['embedding'] = health_df['question'].progress_apply(lambda x: model.encode(x))

In [ ]:
def preprocess_text(text, stop_words):
    tokens = okt.morphs(text)  # 형태소 분석으로 토큰화
    filtered_tokens = [word for word in tokens if word not in stop_words]  # 불용어 제거
    return ' '.join(filtered_tokens)

## 7. 사용자의 텍스트 모욕어 감지 및 유사한 대답 내놓기

#### 7-1. 사용자가 내놓은 질문과 챗봇 데이터 질문이 어느정도 유사한지 계산한 후 가장 유사한 질문에 대응되는 대답 데이터를 챗봇 데이터가 불러오게 합니다.

#### 7-2.  그러나, 만약 사용자가 질문했던 것 중 가장 높은 유사도 값이 0.8이하라면 대답 값을 내놓지 않고 "죄송하지만, 이해할 수 없습니다."라는 대답을 내놓게 됩니다.

In [ ]:
def get_best_response(user_input, model, public_df, heal_df, threshold=0.8):
    user_embedding = model.encode(user_input)  # 사용자 입력 임베딩
    best_similarity = 0
    best_response = None

    # 모든 데이터프레임에서 유사도 계산
    for df in [public_df, heal_df]:  # 리스트에 데이터프레임 추가
        for _, row in df.iterrows():
            similarity = util.cos_sim(user_embedding, row['embedding'])[0][0].item()
            if similarity > best_similarity:
                best_similarity = similarity
                best_response = row['answer']  # 가장 유사한 답변 업데이트

    # 임계값 이상의 유사도일 경우 답변 반환
    if best_similarity >= threshold:
        return best_response
    else:
        return "죄송하지만, 이해할 수 없습니다."

#### 7-3. 모욕어 라벨이 1로 설정돼 있고 만약 사용자 질문에 라벨 "1"이 부여됐다면, 모욕어 감지가 나오게끔 작동

In [ ]:
def filter_user_input(user_input, clf, model, public_df, heal_df):
    processed_input = preprocess_text(user_input, stop_words)
    user_embedding = model.encode(processed_input)
    
    predicted_label = clf.predict([user_embedding])[0]
    
    if predicted_label == 1:
        return "경고: 모욕적인 텍스트가 감지되었습니다."
    else:
        # 정상 텍스트인 경우 최적의 질문과 답변 선택
        return get_best_response(processed_input, model, public_df, heal_df)  # 두 데이터프레임 전달

In [ ]:
# 사용자 입력 테스트
while True:
    user_input = input("사용자 입력 (종료하려면 'exit' 입력): ")
    if user_input.lower() == 'exit':
        break
    
    result = filter_user_input(user_input, clf, model)
    print(result)